# Programming in Bioinformatics and Systems Biology
# Master in Omics Data Analysis and Systems Biology
# Course 2023/2024
## Alignment Algorithms in Dynamic Programming

# Introduction: The Smith-Waterman algorithm

The **Smith-Waterman** algorithm solves the local alignment of two sequences by applying dynamic programming. It is a variant of the **Needleman-Wunsch** algorithm.

## Algorithm input

As input to the algorithm we have:

* Sequence **s** to align.
* Sequence **t** to be aligned.
* A substitution matrix **m**.
* Gap penalty **gap_penalty**.


## Data structures and recursive relations


The following describes the data structures and recursive relationships used in the algorithm:

### Data structures

 
* Decision matrix **dec**

These matrices have as many rows as (len(**s**)+1) and as many columns as (len(**t**)+1).

### Initialisation

* The first row and first column of **punt** are initialised to the value *0*.
* The first row and first column of **dec** are initialised to the value *4*.
* **dec[0][0]** is initialized to *0*.

### Recursive relationships

* For all 1 <= i < len(s), 1<= j < len(t):
  - punt[i][j] receives the maximum of among the following possibilities:
    + punt[i-1][j-1] + m[(s[i-1],t[j-1])]
    + punt[i-1][j] + gap_penalty
    + punt[i][j-1] + gap_penalty
    + 0
  - dec[i][j] takes the maximum argument (np.argmax) from the above possibilities.

## Value of the optimal local alignment

The maximum is located in the **point** array, that will be the value of the optimal local alignment. 

## Inverse trace

To reconstruct the alignment, an inverse trace is performed on the **dec** matrix from the position of the maximum in the **punt** matrix. The inverse trace is performed following the procedure seen in theory class. The aligned sequences are called **sp** and **tp**, which correspond respectively to the sequence **s** and the sequence **t** after having introduced the corresponding gaps (dash symbol **-**) according to the inverse trace.


## Algorithm output

The algorithm shall return in a tuple:

* The aligned sequence **sp**.
* The aligned sequence **tp**.
* The value of the obtained alignment.




In [1]:
from Bio.Align import substitution_matrices

In [2]:
# I load the substitution matrix BLOSUM62 into a new variable from the substitution_matrices subpackage of Bio.Align.
blosum62 = substitution_matrices.load("BLOSUM62")

## Exercise 1

Let's implement the **Needleman-Wunsch** algorithm in order to make comparisons between global and local alignment. 

Implement the **globalAlignment** function that receives:

* Sequence **s** to align.
* Sequence **t** to align.
* A substitution matrix **m**.
* Gap penalty **gap_penalty**.

The function shall perform **global alignment by implementing the Needleman-Wunsch algorithm** and return in a tuple:

* The aligned sequence **sp**.
* The aligned sequence **tp**.
* The value of the obtained alignment.

It is allowed to use code from the practices of the course and it is allowed (and recommended) to implement the auxiliary functions that are deemed appropriate. 

Note: It would not be valid to use *pairwise2.global*.

```
In:
globalAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

Out:
('VIVALA-VIDA', 'VIVALADAVIS', 21.0)
```


In [3]:
import numpy as np

In [4]:
def getGlobalTables(s, t, m, gap_penalty):
    
    punt = [] # Score array to store the cumulative score for each position
    dec = [] # Decision matrix to store the directions taken during alignment
    
    globalvalue = 0
    
    for i in range(0, len(s) + 1): # Initialise punt and dec tables with zeros
        punt.append([0] * (len(t) + 1))
        dec.append([0] * (len(t) + 1))
        
    g = 0
    
    for i in range(0, len(s) + 1): # I initialise the first columns with gap penalties
        punt[i][0] = g
        g += gap_penalty
        dec[i][0] = 2
        
    g = 0
    
    for j in range(0, len(t) + 1): # # I initialise the first rows with penalty for gaps
        punt[0][j] = g
        g += gap_penalty
        dec[0][j] = 3
        
    dec[0][0] = 0
    
    for i in range(0, len(s)): # With these nested loops I fill the score and pivot tables with the corresponding values
        for j in range(0, len(t)): 
            value1 = punt[i][j] + m[(s[i], t[j])]
            value2 = punt[i][j+1] + gap_penalty
            value3 = punt[i+1][j] + gap_penalty
            values = [value1, value2, value3]
            globalvalue = punt[i+1][j+1] = max(values) # To store the total value of the global alignment
            dec[i+1][j+1] = np.argmax(values) + 1
            
    return (punt, dec, globalvalue)

getGlobalTables('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

([[0, -4, -8, -12, -16, -20, -24, -28, -32, -36, -40, -44],
  [-4, 4.0, 0.0, -4.0, -8.0, -12.0, -16.0, -20.0, -24.0, -28.0, -32.0, -36.0],
  [-8, 0.0, 8.0, 4.0, 0.0, -4.0, -8.0, -12.0, -16.0, -20.0, -24.0, -28.0],
  [-12, -4.0, 4.0, 12.0, 8.0, 4.0, 0.0, -4.0, -8.0, -12.0, -16.0, -20.0],
  [-16, -8.0, 0.0, 8.0, 16.0, 12.0, 8.0, 4.0, 0.0, -4.0, -8.0, -12.0],
  [-20, -12.0, -4.0, 4.0, 12.0, 20.0, 16.0, 12.0, 8.0, 4.0, 0.0, -4.0],
  [-24, -16.0, -8.0, 0.0, 8.0, 16.0, 24.0, 20.0, 16.0, 12.0, 8.0, 4.0],
  [-28, -20.0, -12.0, -4.0, 4.0, 12.0, 20.0, 21.0, 20.0, 20.0, 16.0, 12.0],
  [-32, -24.0, -16.0, -8.0, 0.0, 8.0, 16.0, 17.0, 20.0, 23.0, 24.0, 20.0],
  [-36, -28.0, -20.0, -12.0, -4.0, 4.0, 12.0, 22.0, 18.0, 19.0, 20.0, 24.0],
  [-40, -32.0, -24.0, -16.0, -8.0, 0.0, 8.0, 18.0, 26.0, 22.0, 18.0, 21.0]],
 [[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
  [2, 1, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3],
  [2, 2, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3],
  [2, 1, 2, 1, 3, 3, 3, 3, 3, 1, 3, 3],
  [2, 2, 2, 2, 1, 3, 1, 3, 1, 3,

In [5]:
def getGlobalTrace(dec):
    
    i = len(dec) - 1
    j = len(dec[0]) - 1
    trace = []
    
    while dec[i][j] != 0: # I traverse the trajectory backwards from the last position to the starting position.
        trace.append(dec[i][j])
        
        if dec[i][j] == 1: # For diagonal movements
            i -= 1
            j -= 1
        elif dec[i][j] == 2: # For vertical movements
            i -= 1
        else: # This would be dec[i][j] == 3, which means horizontal movement
            j -= 1
            
    trace.reverse() # Reverse the final trace to use it in the correct order
        
    return trace

In [6]:
def globalAlignment(s, t, m, gap_penalty):
    
    punt, dec, globalvalue = getGlobalTables(s, t, m, gap_penalty)
    trace = getGlobalTrace(dec)
    
    i = 0
    j = 0
    sp = ""
    tp = ""
    
    for k in range(0, len(trace)): # I reconstruct the aligned sequences based on their trajectory
        if trace[k] == 1:
            sp += s[i]
            tp += t[j]
            i += 1
            j += 1
        elif trace[k] == 2:
            sp += s[i]
            tp += '-'
            i += 1
        else:
            sp += '-'
            tp += t[j]
            j += 1
            
    return (sp, tp, globalvalue)
    
globalAlignment('VIVALAVIDA', 'VIVALADAVIS', blosum62, -4)

('VIVALA-VIDA', 'VIVALADAVIS', 21.0)

## Exercise 2

Implement the **localAlignment** function that receives:

* Sequence **s** to align.
* Sequence **t** to align.
* A substitution matrix **m**.
* Gap penalty **gap_penalty**.


The function shall perform **local alignment by implementing the Smith-Waterman** algorithm and return in a tuple:

* The aligned sequence **sp**.
* The aligned sequence **tp**.
* The value of the obtained alignment.

It is allowed to use code from the practices of the course and it is allowed (and recommended) to implement the auxiliary functions that are deemed appropriate. 

Note: It would not be valid to use *pairwise2.local*.

```
In:
localAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

Out:
('VIVALAVIDA', 'VIVALA--DA', 26.0)
```


In [7]:
def getLocalTables(s, t, m, gap_penalty):
    
    punt = []
    dec = []
    
    for i in range(0, len(s) + 1):
        punt.append([0] * (len(t) + 1))
        dec.append([0] * (len(t) + 1))
    
    for i in range(0, len(s) + 1): # The difference with the getGlobalTables code is that here I do not apply the gap penalties
        punt[i][0] = 0
        dec[i][0] = 4
    
    for j in range(0, len(t) + 1): # I do not apply penalties here either
        punt[0][j] = 0
        dec[0][j] = 4
        
    dec[0][0] = 0
    
    for i in range(0, len(s)): 
        for j in range(0, len(t)): 
            
            value1 = punt[i][j] + m[(s[i], t[j])]
            value2 = punt[i][j+1] + gap_penalty
            value3 = punt[i+1][j] + gap_penalty
            value4 = 0
            values = [value1, value2, value3, value4]
            punt[i+1][j+1] = max(values)
            dec[i+1][j+1] = np.argmax(values) + 1
            
    return (punt, dec)

In [8]:
def getLocalValue(punt):
    
    localvalue = 0
    i_max = 0
    j_max = 0
    
    for i in range(0, len(punt)):
        for j in range(0, len(punt[0])):
            if punt[i][j] > localvalue:
                localvalue = punt[i][j] # I store the maximum value of the scorecard in the variable localvalue
                i_max = i # Co-ordinates of the position corresponding to the maximum value
                j_max = j
                
    return (localvalue, i_max, j_max)

In [9]:
def getLocalTrace(dec, punt):
    
    _, i, j = getLocalValue(punt) # In this function I use the coordinates of the maximum value obtained before to make the trace
    trace = []
    
    while dec[i][j] != 0:
        trace.append(dec[i][j])
        
        if dec[i][j] == 1:
            i -= 1
            j -= 1
        elif dec[i][j] == 2:
            i -= 1
        else:
            j -= 1
            
    trace.reverse()
    
    return trace

In [10]:
def localAlignment(s, t, m, gap_penalty):
    
    punt, dec = getLocalTables(s, t, m, gap_penalty)
    trace = getLocalTrace(dec, punt)
    localvalue, _, _ = getLocalValue(punt)
    
    i = 0
    j = 0
    sp = ""
    tp = ""
    
    for k in range(0, len(trace)):
        if trace[k] == 1:
            sp += s[i]
            tp += t[j]
            i += 1
            j += 1
        elif trace[k] == 2:
            sp += s[i]
            tp += "-"
            i += 1
        elif trace[k] == 3:
            tp += "-"
            sp += t[j]
            j += 1
        else:
            sp += "-"
            tp += "-"
            
    return (sp, tp, localvalue)
    
localAlignment('VIVALAVIDA', 'VIVALADAVIS', blosum62, -4)

('VIVALAVIDA', 'VIVALA--DA', 26.0)

## Exercise 3

Write a **formatAlignment** function that receives a tuple representing an alignment as produced by the functions in the previous exercises and prints the alignment on the screen in an aesthetically pleasing way.

```
In:
alignment = globalAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)
formatAlignment(alignment)

Out:
VIVALA-VIDA
|||||| ....
VIVALADAVIS
Score: 21.0

In:
alignment = localAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)
formatAlignment(alignment)

Out:
VIVALAVIDA
||||||  ||
VIVALA--DA
Score: 26.0
```

In [11]:
def formatAlignment(alignment):
    
    sp, tp, score = alignment
    
    for letter in sp: 
        print(letter, end = '') # I show the first aligned sequence
    
    print()
    
    for i in range(len(sp)): # I display the alignment characters according to whether the letters match, do not match or there is a "-"
        if sp[i] == tp[i]:
            print('|', end = '')
        elif sp[i] == '-' or tp[i] == '-' :
            print(' ', end = '')
        else:
            print('.', end = '')
    
    print()
    
    for letter in tp:
        print(letter, end = '') # I show the second aligned sequence
        
    print()
    
    print("Score:", score)
    
    print()

# Example of use with global alignment
alignment = globalAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)
formatAlignment(alignment)

# Example of use with local alignment
alignment_local = localAlignment('VIVALAVIDA', 'VIVALADAVIS', blosum62, -4)
formatAlignment(alignment_local)

VIVALA-VIDA
|||||| ....
VIVALADAVIS
Score: 21.0

VIVALAVIDA
||||||  ||
VIVALA--DA
Score: 26.0



## Exercise 4

Previous steps (it is not necessary to do them with Python):

1. Download from NCBI the sequence of the Glycoprotein S (Spike Protein) of the Covid-2 virus in a FASTA file.

2. Repeat the operation for S-protein sequences of other coronaviruses.

Generate a multi-fasta file with all S-protein sequences. The description of each sequence should indicate the virus used and the accession number. Submit this file together with the rest of the task.


You are asked to (in Python):

* Read the generated multi-fasta file
* Test the implemented functions **globalAlignment** and **localAlignment** by comparing the Covid2 sequence with that of the other viruses. Use different parameters for the substitution matrix and penalty for gaps.
* Test the corresponding functions of the **pairwise2** package to perform the same alignments.
* Comment on the results.

In [12]:
from Bio import Entrez, Seq, SeqIO

In [13]:
# I load the multifasta file and read it
multi_fasta = list(SeqIO.parse("multifasta.fasta", "fasta")) 

for e in multi_fasta:
    print (e.description)
    print (e.seq)
    print ()

YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQN

In [14]:
# I load the substitution matrix PAM250 into a new variable from the substitution_matrices subpackage of Bio.Align.
pam250 = substitution_matrices.load("PAM250")

In [15]:
# Importing the package
# https://biopython.org/docs/latest/api/Bio.pairwise2.html
from Bio import pairwise2

# Import format_alignment method to display alignments on screen
from Bio.pairwise2 import format_alignment

In [ ]:
# As can be seen below for each of the different conditions and comparisons the functions of the pairwise2 package
# give the same alignment scores as when using the globalAlignment and localAlignment functions implemented by me,
# which leads me to the conclusion that I have performed these functions correctly.

In [16]:
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", globalAlignment(multi_fasta[0].seq, multi_fasta[e].seq, pam250, -4)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.globalds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = pam250, open=-4, extend=-4, score_only=True))
    print ()

# As can be seen in the results, the comparison with the highest score is for the bat coronavirus, which means that it has a 
# higher similarity to the Covid-2 protein S sequence.

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2021.0

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2188.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2021.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2188.0



In [17]:
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", globalAlignment(multi_fasta[0].seq, multi_fasta[e].seq, blosum62, -4)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.globalds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = blosum62, open=-4, extend=-4, score_only=True))
    print ()
    
# In this case, the S protein with the highest similarity is again from the same coronavirus, the bat coronavirus, as in the 
# rest of the conditions for both the local and global alignment, the substitution matrices pam250 and blosum62, and the 
# different values for the gap substitutions.

# These results make sense because, as we have seen, these two proteins are the most similar and are more likely to obtain 
# higher scores for both global and local alignments.

# On the other hand, we can see the change in scores when using different substitution matrices, with pam250 getting the 
# highest scores. This is due to the fact that the blosum62 matrix is used for sequences with less evolutionary divergence 
# time than the pam250, and it is characterised by being a more restrictive and specific matrix, penalizing more substitutions
# between amino acids that are less common in conserved sequences.

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 1544.0

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 1783.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 1544.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 1783.0



In [18]:
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", globalAlignment(multi_fasta[0].seq, multi_fasta[e].seq, blosum62, -2)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Overall comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.globalds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = blosum62, open=-2, extend=-2, score_only=True))
    print ()
    
# If you look at it, the difference with the previous functions is that we now decrease the value of the penalties, which 
# leads to an increase in the scores.

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2117.0

Overall comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2301.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2117.0

Overall comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2301.0



In [19]:
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", localAlignment(multi_fasta[0].seq, multi_fasta[e].seq, pam250, -4)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.localds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = pam250, open=-4, extend=-4, score_only=True))
    print ()

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2032.0

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2226.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2032.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2226.0



In [20]:
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", localAlignment(multi_fasta[0].seq, multi_fasta[e].seq, blosum62, -4)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.localds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = blosum62, open=-4, extend=-4, score_only=True))
    print ()

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 1550.0

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 1832.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 1550.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 1832.0



In [21]:
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", localAlignment(multi_fasta[0].seq, multi_fasta[e].seq, blosum62, -2)[2])
    print ()
    
for e in range(1, len(multi_fasta)):
    print ("Local comparison of the sequences using pairwise2:", multi_fasta[0].description, "vs", multi_fasta[e].description)
    print ("Score:", pairwise2.align.localds(multi_fasta[0].seq, multi_fasta[e].seq, match_dict = blosum62, open=-2, extend=-2, score_only=True))
    print ()
    
# As can be seen from the comparison between the same conditions in local and global alignments, in general, in almost all 
# conditions tested, the scores of the local alignments are higher than those of the global alignments. This may be because 
# local alignments tend to give higher scores when comparing sequences with highly conserved regions, indicating that the S 
# protein has highly conserved regions among the different types of coronavirus compared.

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2122.0

Local comparison of the sequences: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2316.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_173238.1 spike glycoprotein [Human coronavirus HKU1]
Score: 2122.0

Local comparison of the sequences using pairwise2: YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2] vs YP_001039953.1 spike glycoprotein [Tylonycteris bat coronavirus HKU4]
Score: 2316.0

